In [2]:
# import time

In [3]:
from ebaysdk.finding import Connection
import requests

In [4]:
from ebaysdk.shopping import Connection as Shopping

In [5]:
import pandas as pd 
import  json
import numpy as np
import re
import preprocess_ddey117 as pp
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image


import seaborn as sns 

In [ ]:
# #create function for organizing API call
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['itemId', 'title', 'galleryURL', 
#         'viewItemURL', 'autoPay', 'postalCode', 
#         'sellingStatus', 'shippingInfo', 'listingInfo',
#         'returnsAccepted', 'condition', 'topRatedListing',
#         'galleryPlusPictureURL']
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
#             results.append(prepared_data)
    
       
#         # Add to list if all values are present
# #         if all(prepared_data.values()):
# #             results.append(prepared_data)
    
    
#     return results

In [ ]:
# def prepare_df(df):
#     price_list = []
#     for row in full_dataset:
#         listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['value'])
#         price_list.append(listed_price)

#     df['price_in_US'] = price_list
#     #pull shipping cost from json dict with regex 
#     df['shipping_cost'] = df['shippingInfo'].apply(lambda x: re.findall("(\d+\S+\d)", json.dumps(x)))
#     df['shipping_cost'] = df['shipping_cost'].apply(lambda x: ''.join(x))
#     df.drop(df[df['shipping_cost'] == ''].index, inplace=True)
#     df['shipping_cost'] = df['shipping_cost'].apply(lambda x: np.float(x))

#     #create new feature 'converted price'
#     df['converted_price'] = df['shipping_cost'] + df['price_in_US']
#     df.drop_duplicates(subset=['itemId'],  keep='first', inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     display(df.head())
#     display(df.info())
#     return df

In [6]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
# from base64 import b64encode


In [ ]:
# #this function was used at first to check accuracy of calls using ebay API
# #the less complex "prepare_brands" function seems sufficient enough
# #it is redudant to use regex with working ebay API call

# def modified_prepare_brands(df, bucket_dict_position):

#     df.title = df.title.apply(str.lower)

#     #remove special characters
#     df.title.apply(pp.remove_special_chars)

#     pattern = re.compile(list(bucket_dict.keys())[bucket_dict_position])
#     df[list(bucket_dict.keys())[bucket_dict_position]] = df_bench.title.apply(lambda x: re.sub(pattern, x, str(list(bucket_dict.values())[bucket_dict_position])))

#     df[list(bucket_dict.keys())[bucket_dict_position]] = df[list(bucket_dict.keys())[bucket_dict_position]].apply(lambda x: np.float(x))
    
#     df['profit'] = (df['converted_price'] - df[list(bucket_dict.keys())[bucket_dict_position]] - overhead_cost)
#     df['ROI'] = (df['profit']/(df[list(bucket_dict.keys())[bucket_dict_position]] + overhead_cost))*100.0

#     return df

In [7]:
def prepare_df(df):
    price_list = []
    ship_price_list = []
    condition_list = []
    condition = None
    for row in full_dataset:
        listed_price = float(row['sellingStatus']['convertedCurrentPrice']['value'])
        price_list.append(listed_price)
     
        try:
            listed_ship_price = float(row['shippingInfo']['shippingServiceCost']['value'])
            ship_price_list.append(listed_ship_price)
        except: 
            listed_ship_price = 0
            ship_price_list.append(listed_ship_price)

        try:
            condition = float(row['condition']['conditionId'])
            condition_list.append(condition)
        except: 
            conditon = 0
            condition_list.append(condition)

    df['shipping_cost'] = ship_price_list
    df['price_in_US'] = price_list
    df['condition'] = condition_list
    
    #create new feature 'converted price'
    df['converted_price'] = df['shipping_cost'] + df['price_in_US']
    df.drop_duplicates(subset=['itemId'],  keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


In [8]:
# request = {
#            'categoryId': 48818,
#             'itemFilter': [
#                             {'name': 'Condition', 'value': 'Used'},
#                             {'name': 'ListingType', 'value': 'FixedPrice'}
#                           ],
#             'aspectFilter': [
#                               {'aspectName': 'Brand', 'aspectValueName': 'Benchmade'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'Buck'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'Case'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'Kershaw'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'Victorinox'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'CRKT'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'Leatherman'},
#                               {'aspectName': 'Brand', 'aspectValueName': 'Spyderco'},
#                              ],
#             'paginationInput': {
#                                 'entriesPerPage': 100,
#                                 'pageNumber': 1

#                                 }}

In [9]:
bucket_dict = {'benchmade': 45.0,
               'buck': 20.0,
               'case': 20.0,
               'crkt': 15.0,
               'kershaw': 15.0,
               'leatherman': 30.0,
               'sog': 15.0,
               'spyderco': 30.0,
               'victorinox': 20.0
              }

overhead_cost = 3
def prepare_brands(df, bucket_dict_position):

    df.title = df.title.apply(str.lower)
 
    #remove special characters
    df.title.apply(pp.remove_special_chars)
    df['brand'] = str(list(bucket_dict.keys())[bucket_dict_position])
    df['cost'] = float(list(bucket_dict.values())[bucket_dict_position])
    df['profit'] = (df['converted_price'] -  df['cost'] - overhead_cost)
    df['ROI'] = (df['profit']/( df['cost'] + overhead_cost))*100.0
    
    return df

def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['itemId', 'title', 'galleryURL', 
        'viewItemURL', 'autoPay', 'postalCode', 
        'sellingStatus', 'shippingInfo', 'listingInfo',
        'returnsAccepted', 'condition', 'topRatedListing',
        'galleryPlusPictureURL']
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
            results.append(prepared_data)
    
       
        # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
    
    return results

def knife_request(Brand, dict_pos):
    api = Connection(config_file='ebay.yaml', debug=False, siteid="EBAY-US")

    request = {
                'categoryId': 48818,
                'itemFilter': [
    #                                 {'name': 'Condition', 'value': 'Used'},
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [
                                  {'aspectName': 'Brand', 'aspectValueName': Brand}],


                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': 1

                                    },

                }

    #     request['paginationInput']['pageNumber'] = page

    response = api.execute('findItemsAdvanced', request)


    response_pages = response.dict()

    full_dataset = []
    
    total_pages = int(response_pages['paginationOutput']['totalPages'])

    if total_pages > 100:
        pages_to_request = 100
        
    else:
        pages_to_request = total_pages - 1
        
        

    for page in range(1, pages_to_request):
        # Add or update the "offset" key-value pair in url_params

        # Make the query and get the response

        api = Connection(config_file='ebay.yaml', debug=False, siteid="EBAY-US")

        request = {
                    'categoryId': 48818,
                    'itemFilter': [
    #                                 {'name': 'Condition', 'value': 'Used'},
                                    {'name': 'ListingType', 'value': 'FixedPrice'}
                                  ],
                    'aspectFilter': [
                                      {'aspectName': 'Brand', 'aspectValueName': Brand}],


                    'paginationInput': {
                                        'entriesPerPage': 100,
                                        'pageNumber': page

                                        },

                    }


        response = api.execute('findItemsAdvanced', request)

        #save the response as a json dict
        response_dict = response.dict()


        #index dict to appropriate index
        results_list_of_dicts = response_dict['searchResult']['item']

        # Call the prepare_data function to get a list of processed data
        prepared_knives = prepare_data(results_list_of_dicts)

        # Extend full_dataset with this list (don't append, or you'll get
        # a list of lists instead of a flat list)
        full_dataset.extend(prepared_knives)

    # Check the length of the full dataset. It will be up to `total`,
    # potentially less if there were missing values
    display(len(full_dataset))
    
    df = pd.DataFrame(full_dataset)
    
    price_list = []
    ship_price_list = []
    condition_list = []
    condition = None
    for row in full_dataset:
        try:
            listed_price = float(row['sellingStatus']['convertedCurrentPrice']['value'])
            price_list.append(listed_price)
        except:
            listed_price = "Na"
            price_list.append(listed_price)
        try:
            listed_ship_price = float(row['shippingInfo']['shippingServiceCost']['value'])
            ship_price_list.append(listed_ship_price)
        except: 
            listed_ship_price = 0
            ship_price_list.append(listed_ship_price)
        try:
            condition = float(row['condition']['conditionId'])
            condition_list.append(condition)
        except: 
            conditon = 0
            condition_list.append(condition)

    df['shipping_cost'] = ship_price_list
    df['price_in_US'] = price_list
    df['condition'] = condition_list
    
    #create new feature 'converted price'
    df['converted_price'] = df['shipping_cost'] + df['price_in_US']
    df.drop_duplicates(subset=['itemId'],  keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df = prepare_brands(df, dict_pos)
    
    return df

In [11]:
bench_df = knife_request('Benchmade', 0)

25870

In [12]:
bench_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1974 entries, 0 to 1973
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 1974 non-null   object 
 1   title                  1974 non-null   object 
 2   galleryURL             1972 non-null   object 
 3   viewItemURL            1974 non-null   object 
 4   autoPay                1974 non-null   object 
 5   postalCode             1862 non-null   object 
 6   sellingStatus          1974 non-null   object 
 7   shippingInfo           1974 non-null   object 
 8   listingInfo            1974 non-null   object 
 9   returnsAccepted        1974 non-null   object 
 10  condition              1974 non-null   float64
 11  topRatedListing        1974 non-null   object 
 12  galleryPlusPictureURL  236 non-null    object 
 13  shipping_cost          1974 non-null   float64
 14  price_in_US            1974 non-null   float64
 15  conv

In [13]:
bench_df.to_csv('data/df_bench1.csv', index=False)

### Domain Understading: Cost Breakdown
- padded envelopes: \$0.50 per knife
- flatrate shipping: \$4.45 per knife
- brand knife at surplus store: 15, 20, 30, or 45 dollars per knife
- overhead expenses (gas, cleaning suplies, sharpening supplies, etc): $7

In [14]:
bucket_dict

{'benchmade': 45.0,
 'buck': 20.0,
 'case': 20.0,
 'crkt': 15.0,
 'kershaw': 15.0,
 'leatherman': 30.0,
 'sog': 15.0,
 'spyderco': 30.0,
 'victorinox': 20.0}

In [15]:
buck_df = knife_request('Buck', 1)

39000

In [16]:
buck_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 2940 non-null   object 
 1   title                  2940 non-null   object 
 2   galleryURL             2937 non-null   object 
 3   viewItemURL            2940 non-null   object 
 4   autoPay                2940 non-null   object 
 5   postalCode             2896 non-null   object 
 6   sellingStatus          2940 non-null   object 
 7   shippingInfo           2940 non-null   object 
 8   listingInfo            2940 non-null   object 
 9   returnsAccepted        2940 non-null   object 
 10  condition              2940 non-null   float64
 11  topRatedListing        2940 non-null   object 
 12  galleryPlusPictureURL  347 non-null    object 
 13  shipping_cost          2940 non-null   float64
 14  price_in_US            2940 non-null   float64
 15  conv

In [17]:
buck_df.to_csv('data/df_buck.csv', index=False)

In [18]:
case_df = knife_request('Case', 2)

128700

In [19]:
case_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9684 entries, 0 to 9683
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 9684 non-null   object 
 1   title                  9684 non-null   object 
 2   galleryURL             9427 non-null   object 
 3   viewItemURL            9684 non-null   object 
 4   autoPay                9684 non-null   object 
 5   postalCode             9515 non-null   object 
 6   sellingStatus          9684 non-null   object 
 7   shippingInfo           9684 non-null   object 
 8   listingInfo            9684 non-null   object 
 9   returnsAccepted        9684 non-null   object 
 10  condition              9684 non-null   float64
 11  topRatedListing        9684 non-null   object 
 12  galleryPlusPictureURL  4234 non-null   object 
 13  shipping_cost          9684 non-null   float64
 14  price_in_US            9684 non-null   float64
 15  conv

In [20]:
case_df.to_csv('data/df_case.csv', index=False)

In [21]:
df_caseXX = knife_request('Case XX', 2)

107900

In [22]:
df_caseXX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8077 entries, 0 to 8076
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 8077 non-null   object 
 1   title                  8077 non-null   object 
 2   galleryURL             8070 non-null   object 
 3   viewItemURL            8077 non-null   object 
 4   autoPay                8077 non-null   object 
 5   postalCode             7533 non-null   object 
 6   sellingStatus          8077 non-null   object 
 7   shippingInfo           8077 non-null   object 
 8   listingInfo            8077 non-null   object 
 9   returnsAccepted        8077 non-null   object 
 10  condition              8077 non-null   float64
 11  topRatedListing        8077 non-null   object 
 12  galleryPlusPictureURL  2056 non-null   object 
 13  shipping_cost          8077 non-null   float64
 14  price_in_US            8077 non-null   float64
 15  conv

In [23]:
df_caseXX.to_csv('data/df_CaseXX.csv', index=False)

In [24]:
bucket_dict 

{'benchmade': 45.0,
 'buck': 20.0,
 'case': 20.0,
 'crkt': 15.0,
 'kershaw': 15.0,
 'leatherman': 30.0,
 'sog': 15.0,
 'spyderco': 30.0,
 'victorinox': 20.0}

In [25]:
df_crkt = knife_request("CRKT", 3)

19500

In [26]:
df_crkt.to_csv('data/df_crkt.csv', index=False)

In [27]:
df_kershaw = knife_request('Kershaw', 4)

KeyError: 'item'

In [33]:
full_dataset = []
for page in range(1, 58):
#         # Add or update the "offset" key-value pair in url_params

#         # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=False, siteid="EBAY-US")

    request = {
                'categoryId': 48818,
                'itemFilter': [
    #                                 {'name': 'Condition', 'value': 'Used'},
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [
                                  {'aspectName': 'Brand', 'aspectValueName': 'Kershaw'}],


                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page

                                    },

                        }

        #     request['paginationInput']['pageNumber'] = page

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()

    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

    # Check the length of the full dataset. It will be up to `total`,
    # potentially less if there were missing values

    df = pd.DataFrame(full_dataset)

In [36]:
df_kershaw = prepare_df(df)

,itemId,title,galleryURL,viewItemURL,autoPay,postalCode,sellingStatus,shippingInfo,listingInfo,returnsAccepted,condition,topRatedListing,galleryPlusPictureURL,shipping_cost,price_in_US,converted_price
0,234780801512,Kershaw Tarheel 1364 Tactical Liner Lock Foldi...,https://i.ebayimg.com/thumbs/images/g/zJcAAOSw...,https://www.ebay.com/itm/Kershaw-Tarheel-1364-...,false,373**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,true,None,0.0,18.69,18.69
1,283684293533,1660 Kershaw Leek Knife silver plain Blade New...,https://i.ebayimg.com/thumbs/images/g/VQ0AAOSw...,https://www.ebay.com/itm/1660-Kershaw-Leek-Kni...,true,982**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,true,None,0.0,41.95,41.95
2,195454753696,Kershaw Damascus Skyline 1760DAM! Discontinued...,https://i.ebayimg.com/thumbs/images/g/itIAAOSw...,https://www.ebay.com/itm/Kershaw-Damascus-Skyl...,false,923**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,true,None,0.0,100.00,100.00
3,275535106552,Kershaw Zing Framelock Knife Folding Blade Sta...,https://i.ebayimg.com/thumbs/images/g/grcAAOSw...,https://www.ebay.com/itm/Kershaw-Zing-Frameloc...,true,125**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,false,None,0.0,19.99,19.99
4,175006871402,"Kershaw Kuro Assisted Folding Knife 3.1"" 8Cr13...",https://i.ebayimg.com/thumbs/images/g/2o4AAOSw...,https://www.ebay.com/itm/Kershaw-Kuro-Assisted...,true,243**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,true,https://galleryplus.ebayimg.com/ws/web/1750068...,0.0,25.39,25.39


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5587 entries, 0 to 5586
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 5587 non-null   object 
 1   title                  5587 non-null   object 
 2   galleryURL             5586 non-null   object 
 3   viewItemURL            5587 non-null   object 
 4   autoPay                5587 non-null   object 
 5   postalCode             5452 non-null   object 
 6   sellingStatus          5587 non-null   object 
 7   shippingInfo           5587 non-null   object 
 8   listingInfo            5587 non-null   object 
 9   returnsAccepted        5587 non-null   object 
 10  condition              5587 non-null   float64
 11  topRatedListing        5587 non-null   object 
 12  galleryPlusPictureURL  1014 non-null   object 
 13  shipping_cost          5587 non-null   float64
 14  price_in_US            5587 non-null   float64
 15  conv

None

In [38]:
df_kershaw.to_csv('data/df_kershaw.csv', index=False)

In [39]:
df_leatherman = knife_request('Leatherman', 5)

2522

In [40]:
df_leatherman.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194 entries, 0 to 193
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 194 non-null    object 
 1   title                  194 non-null    object 
 2   galleryURL             194 non-null    object 
 3   viewItemURL            194 non-null    object 
 4   autoPay                194 non-null    object 
 5   postalCode             174 non-null    object 
 6   sellingStatus          194 non-null    object 
 7   shippingInfo           194 non-null    object 
 8   listingInfo            194 non-null    object 
 9   returnsAccepted        194 non-null    object 
 10  condition              194 non-null    float64
 11  topRatedListing        194 non-null    object 
 12  galleryPlusPictureURL  24 non-null     object 
 13  shipping_cost          194 non-null    float64
 14  price_in_US            194 non-null    float64
 15  conver

In [41]:
df_leatherman.to_csv('data/df_leatherman.csv', index=False)

In [42]:
df_sog = knife_request('SOG', 6)

16900

In [43]:
df_sog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1282 entries, 0 to 1281
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 1282 non-null   object 
 1   title                  1282 non-null   object 
 2   galleryURL             1282 non-null   object 
 3   viewItemURL            1282 non-null   object 
 4   autoPay                1282 non-null   object 
 5   postalCode             1237 non-null   object 
 6   sellingStatus          1282 non-null   object 
 7   shippingInfo           1282 non-null   object 
 8   listingInfo            1282 non-null   object 
 9   returnsAccepted        1282 non-null   object 
 10  condition              1282 non-null   float64
 11  topRatedListing        1282 non-null   object 
 12  galleryPlusPictureURL  270 non-null    object 
 13  shipping_cost          1282 non-null   float64
 14  price_in_US            1282 non-null   float64
 15  conv

In [44]:
df_sog.to_csv('data/df_sog.csv', index=False)

In [45]:
df_spyderco = knife_request('Spyderco', 7)

103090

In [46]:
df_spyderco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7768 entries, 0 to 7767
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 7768 non-null   object 
 1   title                  7768 non-null   object 
 2   galleryURL             7765 non-null   object 
 3   viewItemURL            7768 non-null   object 
 4   autoPay                7768 non-null   object 
 5   postalCode             7374 non-null   object 
 6   sellingStatus          7768 non-null   object 
 7   shippingInfo           7768 non-null   object 
 8   listingInfo            7768 non-null   object 
 9   returnsAccepted        7768 non-null   object 
 10  condition              7768 non-null   float64
 11  topRatedListing        7768 non-null   object 
 12  galleryPlusPictureURL  1854 non-null   object 
 13  shipping_cost          7768 non-null   float64
 14  price_in_US            7768 non-null   float64
 15  conv

In [47]:
df_spyderco.to_csv('data/df_spyderco.csv', index=False)

In [13]:
# full_dataset = []
# for page in range(1, 58):
#         # Add or update the "offset" key-value pair in url_params

#         # Make the query and get the response

api = Connection(config_file='ebay.yaml', debug=False, siteid="EBAY-US")

request = {
            'categoryId': 48818,
            'itemFilter': [
#                                 {'name': 'Condition', 'value': 'Used'},
                            {'name': 'ListingType', 'value': 'FixedPrice'}
                          ],
            'aspectFilter': [
                              {'aspectName': 'Brand', 'aspectValueName': 'Victorinox'}],


            'paginationInput': {
                                'entriesPerPage': 100,
                                'pageNumber': 1

                                },

                    }

    #     request['paginationInput']['pageNumber'] = page

response = api.execute('findItemsAdvanced', request)

#save the response as a json dict
response_dict = response.dict()

tot_pages = response_dict['paginationOutput']['totalPages']

print(f"total pages: {tot_pages}")

total pages: 97


In [34]:
full_dataset = []
for page in range(1, 86):

    api = Connection(config_file='ebay.yaml', debug=False, siteid="EBAY-US")

    request = {
                'categoryId': 48818,
                'itemFilter': [
    #                                 {'name': 'Condition', 'value': 'Used'},
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [
                                  {'aspectName': 'Brand', 'aspectValueName': 'Victorinox'}],


                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page

                                    },

                        }

    response = api.execute('findItemsAdvanced', request)

    response_dict = response.dict()

    results_list_of_dicts = response_dict['searchResult']['item']

    prepared_knives = prepare_data(results_list_of_dicts)

    full_dataset.extend(prepared_knives)

In [35]:
df = pd.DataFrame(full_dataset)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104533 entries, 0 to 104532
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   itemId                 104533 non-null  object
 1   title                  104533 non-null  object
 2   galleryURL             104507 non-null  object
 3   viewItemURL            104533 non-null  object
 4   autoPay                104533 non-null  object
 5   postalCode             92768 non-null   object
 6   sellingStatus          104533 non-null  object
 7   shippingInfo           104533 non-null  object
 8   listingInfo            104533 non-null  object
 9   returnsAccepted        104533 non-null  object
 10  condition              101725 non-null  object
 11  topRatedListing        104533 non-null  object
 12  galleryPlusPictureURL  16848 non-null   object
dtypes: object(13)
memory usage: 10.4+ MB


In [37]:
df_victorinox = prepare_df(df)

,itemId,title,galleryURL,viewItemURL,autoPay,postalCode,sellingStatus,shippingInfo,listingInfo,returnsAccepted,condition,topRatedListing,galleryPlusPictureURL,shipping_cost,price_in_US,converted_price
0,165614958701,Victorinox Classic SD Mini Swiss Army Pocket K...,https://i.ebayimg.com/thumbs/images/g/V8MAAOSw...,https://www.ebay.com/itm/Victorinox-Classic-SD...,true,338**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",false,3000.0,false,None,3.5,5.00,8.50
1,193932638989,NEW Victorinox Super Tinker Red Multifunction ...,https://i.ebayimg.com/thumbs/images/g/60EAAOSw...,https://www.ebay.com/itm/NEW-Victorinox-Super-...,true,331**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,true,None,0.0,29.99,29.99
2,204160933903,Victorinox Classic SD Mini Swiss Army Pocket K...,https://i.ebayimg.com/thumbs/images/g/7Q8AAOSw...,https://www.ebay.com/itm/Victorinox-Classic-SD...,true,492**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,3000.0,true,None,0.0,9.25,9.25
3,154848639295,Victorinox Swiss Army 54111 Classic SD Red Wit...,https://i.ebayimg.com/thumbs/images/g/pB8AAOSw...,https://www.ebay.com/itm/Victorinox-Swiss-Army...,false,775**,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",false,1000.0,false,None,0.0,20.99,20.99
4,222775594040,SWISS ARMY KNIFE VICTORINOX 91mm SCALES/HANDLE...,https://i.ebayimg.com/thumbs/images/g/NicAAOSw...,https://www.ebay.com/itm/SWISS-ARMY-KNIFE-VICT...,false,None,"{'currentPrice': {'_currencyId': 'USD', 'value...","{'shippingServiceCost': {'_currencyId': 'USD',...","{'bestOfferEnabled': 'false', 'buyItNowAvailab...",true,1000.0,false,None,4.8,2.80,7.60


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 7979 non-null   object 
 1   title                  7979 non-null   object 
 2   galleryURL             7977 non-null   object 
 3   viewItemURL            7979 non-null   object 
 4   autoPay                7979 non-null   object 
 5   postalCode             7079 non-null   object 
 6   sellingStatus          7979 non-null   object 
 7   shippingInfo           7979 non-null   object 
 8   listingInfo            7979 non-null   object 
 9   returnsAccepted        7979 non-null   object 
 10  condition              7979 non-null   float64
 11  topRatedListing        7979 non-null   object 
 12  galleryPlusPictureURL  1285 non-null   object 
 13  shipping_cost          7979 non-null   float64
 14  price_in_US            7979 non-null   float64
 15  conv

None

In [38]:
df_victorinox.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7979 entries, 0 to 7978
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   itemId                 7979 non-null   object 
 1   title                  7979 non-null   object 
 2   galleryURL             7977 non-null   object 
 3   viewItemURL            7979 non-null   object 
 4   autoPay                7979 non-null   object 
 5   postalCode             7079 non-null   object 
 6   sellingStatus          7979 non-null   object 
 7   shippingInfo           7979 non-null   object 
 8   listingInfo            7979 non-null   object 
 9   returnsAccepted        7979 non-null   object 
 10  condition              7979 non-null   float64
 11  topRatedListing        7979 non-null   object 
 12  galleryPlusPictureURL  1285 non-null   object 
 13  shipping_cost          7979 non-null   float64
 14  price_in_US            7979 non-null   float64
 15  conv

In [39]:
df_victorinox.to_csv('data/df_victo.csv', index=False)

In [ ]:
# df_bench = pd.read_csv("data/df_bench.csv")
# df_buck = pd.read_csv("data/df_buck.csv")
# df_case = pd.read_csv("data/df_case.csv")
# df_crkt = pd.read_csv("data/df_crkt.csv")
# df_kershaw = pd.read_csv("data/df_kershaw.csv")
# df_leatherman = pd.read_csv("data/df_leatherman.csv")
# df_spyderco = pd.read_csv("data/df_spyderco.csv")
# df_victorinox = pd.read_csv("data/df_victorinox.csv")

In [ ]:
df = pd.concat([df_bench,df_buck,df_case,df_crkt,df_kershaw,df_leatherman,df_spyderco,df_sog,df_victorinox])

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.to_csv("data/full_dataset2.csv", index=False)

After succesfully going through 10,000 items on ebay's website and extracting everything possible, there is still a little bit more extracting to do from the json dictionary before saving the dataframe again. We need to get the price of the knives out of the nested dictionary in the dataframe as well as the shipping cost. After that, I would like to create a new feature called
**"converted price," which is simply the price of the knife listed on the ebay's website plus shipping.**

In [ ]:
df = pd.read_csv('data/tera_df_prepared.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

In [ ]:
df.info()

In [ ]:
api = Shopping(config_file='ebay.yaml', debug=True, siteid="EBAY-US")
request = {
           'itemID': list(itemIds),
           'IncludeSelector': 'ItemSpecifics'
          }
response = api.execute('GetMultipleItems', request)

response_dict = response.dict()

results_list_of_dicts = response_dict['Item']



In [ ]:
response_dict = response.dict()

    #index dict to appropriate index
# results_list_of_dicts = response_dict['searchResult']['item']

In [ ]:
results_list_of_dicts = response_dict['Item']

In [ ]:
# #create function for organizing API call
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['ItemID', 'Title', 'GalleryURL', 
#                 'ViewItemURLForNaturalSearch', 'ItemSpecifics', 
#                 'Country', 'Location', 'ConvertedCurrentPrice',
#                 'PictureURL']
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
#             results.append(prepared_data)
    
       
# #         Add to list if all values are present
# #         if all(prepared_data.values()):
# #             results.append(prepared_data)
    
#     return results

In [ ]:
# #create function for organizing API call
# def prepare_data(data_list):
#     """
#     This function takes in a list of dictionaries and prepares it
#     for analysis
#     """
    
#     # Make a new list to hold results
#     results = []
    
#     for business_data in data_list:
    
#         # Make a new dictionary to hold prepared data for this business
#         prepared_data = {}
        
#         # Extract name, review_count, rating, and price key-value pairs
#         # from business_data and add to prepared_data
#         # If a key is not present in business_data, add it to prepared_data
#         # with an associated value of None
        
#         keys = ['ItemID', 'Title', 'GalleryURL', 
#                 'ViewItemURLForNaturalSearch', 'ItemSpecifics', 

#                 'PictureURL']
        
#         for key in keys:
#             prepared_data[key] = business_data.get(key, None)
#             results.append(prepared_data)
    
       
# #         Add to list if all values are present
# #         if all(prepared_data.values()):
# #             results.append(prepared_data)
    
#     return results

In [ ]:
# def process_list(my_list):
 
#     api = Shopping(config_file='ebay.yaml', debug=True, siteid="EBAY-US")
#     request = {
#                'itemID': my_list,
#                'IncludeSelector': 'ItemSpecifics'
#               }
#     response = api.execute('GetMultipleItems', request)

    

#     #save the response as a json dict
#     response_dict = response.dict()



#     #index dict to appropriate index
#     results_list_of_dicts = response_dict['Item']

#     # Call the prepare_data function to get a list of processed data
#     prepared_knives = prepare_data(results_list_of_dicts)

#     # Extend full_dataset with this list (don't append, or you'll get
#     # a list of lists instead of a flat list)
#     full_dataset.extend(prepared_knives)
    
#     return full_dataset

In [ ]:
# full_dataset = []
# for i in range(0, len(itemIds), 20):
#     process_list(itemIds[i:i+20])

In [ ]:
# itemIds = list(df.itemId)

In [ ]:
# full_dataset = []
# for i in range(0, len(itemIds), 20):
#     process_list(itemIds[i:i+20])

In [ ]:
# import re

In [ ]:
# pattern = re.compile("Opening Mechanism\s*\S+\S+\s*\S+\S+\s\S(\w+)")

In [ ]:
# def extract_opening_mech(line):
#     pattern = re.compile("Opening Mechanism\s*\S+\S+\s*\S+\S+\s\S(\w+)")

#     if re.findall(pattern,str(line)):

#         match = re.findall(pattern,str(line))[0]

#     else:

#         match = 'NA'
        
#     return match

In [ ]:
# df3['opening_mechanism'] = df3.ItemSpecifics.apply(extract_opening_mech)

In [ ]:
# df3

In [ ]:
# pattern = re.compile("Blade Material\s*\S+\S+\s*\S+\S+\s\S(\w+)")

In [ ]:
# def extract_blade_type(line):
#     pattern = re.compile("Blade Material\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):

#         match = re.findall(pattern,str(line))[0]

#     else:

#         match = 'NA'
        
#     return match

In [ ]:
# df3['blade_type'] = df3.ItemSpecifics.apply(extract_blade_type)

In [ ]:
# df3.ItemSpecifics.sample(10).apply(print)

In [ ]:
# def extract_color(line):
#     pattern = re.compile("Color\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):

#         match = re.findall(pattern,str(line))[0]

#     else:

#         match = 'NA'
        
#     return match

In [ ]:
# df3['color'] = df3.ItemSpecifics.apply(extract_color)

In [ ]:
# def extract_blade_type(line):
#     pattern = re.compile("Blade Type\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):

#         match = re.findall(pattern,str(line))[0]

#     else:

#         match = 'NA'
        
#     return match

In [ ]:
# df3['blade_type'] = df3.ItemSpecifics.apply(extract_blade_type)

In [ ]:
# df3['blade_type'].value_counts()[:20]

In [ ]:
# def extract_manufacture_region(line):
#     pattern = re.compile("Country/Region of Manufacture\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):
#         match = re.findall(pattern,str(line))[0]
#     else:
#         match = 'NA'
#     return match
        


In [ ]:
# df3['region_of_Manufacture'] = df3.ItemSpecifics.apply(extract_manufacture_region)

In [ ]:
# df3['region_of_Manufacture'].value_counts()

In [ ]:
# def extract_handle_material(line):
#     pattern = re.compile("Handle Material\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):
#         match = re.findall(pattern,str(line))[0]
#     else:
#         match = 'NA'
#     return match
        


In [ ]:
# df3['handle_material'] = df3.ItemSpecifics.apply(extract_handle_material)

In [ ]:
# df3['handle_material'].value_counts()[:50]

In [ ]:
# def extract_lock_type(line):
#     pattern = re.compile("Lock Type\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):
#         match = re.findall(pattern,str(line))[0]
#     else:
#         match = 'NA'
#     return match

# df3['lock_type'] = df3.ItemSpecifics.apply(extract_lock_type)

# df3['lock_type'].value_counts()

In [ ]:
# def extract_blade_edge(line):
#     pattern = re.compile("Blade Edge\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):
#         match = re.findall(pattern,str(line))[0]
#     else:
#         match = 'NA'
#     return match
        
# df3['blade_edge'] = df3.ItemSpecifics.apply(extract_blade_edge)

# df3['blade_edge'].value_counts()

In [ ]:
# def extract_dexterity(line):
#     pattern = re.compile("Dexterity\s*\S+\S+\s*\S+\S+\s\S(\w+)")
#     if re.findall(pattern,str(line)):
#         match = re.findall(pattern,str(line))[0]
#     else:
#         match = 'NA'
#     return match
        
# df3['dexterity'] = df3.ItemSpecifics.apply(extract_dexterity)

# df3['dexterity'].value_counts()

In [ ]:
# df3.to_csv('data/item_specifics_df.csv')

In [ ]:
# api = Shopping(config_file='ebay.yaml', debug=True, siteid="EBAY-US")
# request = {
#            'itemID': list(itemIds),
#            'IncludeSelector': 'ItemSpecifics'
#           }
# response = api.execute('GetMultipleItems', request)
# print(response.dict())

In [ ]:
ls

In [ ]:
# from os import listdir
# from PIL import Image

# for imageFolder in listdir('./nn_images2'):
#     try:
#         img = Image.open('./nn_images2/'+imageFolder)
#         img.verify()     # to veify if its an img
#         img.close()     #to close img and free memory space
#     except (IOError, SyntaxError) as e:
#         print('Bad file:', imageFolder)

In [ ]:
# listdir('/nnimages2')

In [ ]:
# root='C:/Users/12108/Documents/GitHub/Neural_Network_Predicting_Reseller_Success_Ebay/nn_images2/'

In [ ]:
# # import pandas as pd
# import matplotlib.pyplot  as plt
# from PIL import Image
# from pathlib import Path
# # import imagesize
# import numpy as np

# # Get the Image Resolutions
# imgs = [img.name for img in Path(root).iterdir() if img.suffix == ".jpg"]
# img_meta = {}
# for f in imgs: img_meta[str(f)] = imagesize.get(root+f)

# # Convert it to Dataframe and compute aspect ratio
# img_meta_df = pd.DataFrame.from_dict([img_meta]).T.reset_index().set_axis(['FileName', 'Size'], axis='columns', inplace=False)
# img_meta_df[["Width", "Height"]] = pd.DataFrame(img_meta_df["Size"].tolist(), index=img_meta_df.index)
# img_meta_df["Aspect Ratio"] = round(img_meta_df["Width"] / img_meta_df["Height"], 2)

# print(f'Total Nr of Images in the dataset: {len(img_meta_df)}')
# img_meta_df.head()

In [ ]:
# # Visualize Image Resolutions

# fig = plt.figure(figsize=(8, 8))
# ax = fig.add_subplot(111)
# points = ax.scatter(img_meta_df.Width, img_meta_df.Height, color='blue', alpha=0.5, picker=True)
# ax.set_title("Image Resolution")
# ax.set_xlabel("Width", size=14)
# ax.set_ylabel("Height", size=14)

In [ ]:
# # Visualize Image Resolutions

# fig = plt.figure(figsize=(8, 8))
# ax = fig.add_subplot(111)
# points = ax.scatter(img_meta_df.Width, img_meta_df.Height, color='blue', alpha=0.5, s=img_meta_df["Aspect Ratio"]*100, picker=True)
# ax.set_title("Image Resolution")
# ax.set_xlabel("Width", size=14)
# ax.set_ylabel("Height", size=14)

```
#Create row for converted Price of Knives in US dollars
price_list = []
for row in full_dataset:
    listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['value'])
    price_list.append(listed_price)
    
df['price_in_US'] = price_list
```

```
#atttempt to pull shipping cost from json dict
shipping_cost_list = []
for row in full_dataset:
    shipping_cost = np.float(row['shippingInfo']['shippingServiceCost']['value'])
    shipping_cost_list.append(shipping_cost)
    
df['shipping_price'] = shipping_cost_list
```

```
#pull shipping cost from json dict with regex 
df['shipping_cost'] = df['shippingInfo'].apply(lambda x: re.findall("(\d+\S+\d)", json.dumps(x)))
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: ''.join(x))
df.drop(df[df['shipping_cost'] == ''].index, inplace=True)
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: np.float(x))

#create new feature 'converted price'
df['converted_price'] = df['shipping_cost'] + df['price_in_US']
df = df.drop_duplicates(subset=['title', 'galleryURL'], keep='first')
display(df.head())
display(df.info())

df.to_csv('data/full_dataset.csv', index=False)
```

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***


# here you run your code to clean the data

```
import code.data_cleaning as dc

full_dataset = dc.full_clean()
```

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***
# here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***


## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***



In [ ]:
# data_knife_dir = 'knife_images'
# data_profit_dir = 'data/profit'
# new_dir = 'split'

In [ ]:
# os.mkdir(new_dir)

In [ ]:
# train_folder = os.path.join(new_dir, 'train')
# train_profit = os.path.join(train_folder, 'profit')
# os.mkdir(train_folder)
# os.mkdir(train_profit)

# test_folder = os.path.join(new_dir, 'test')
# test_profit = os.path.join(test_folder, 'profit')
# os.mkdir(test_folder)
# os.mkdir(test_profit)


# val_folder = os.path.join(new_dir, 'validation')
# val_profit = os.path.join(val_folder, 'profit')
# os.mkdir(val_folder)
# os.mkdir(val_profit)

In [ ]:
# val_profit

In [ ]:
# # train knife regression images
# #80% of data
# imgs = knife_images[:5620]
# for img in imgs:
#     origin = os.path.join(data_knife_dir, img)
#     destination = os.path.join(train_profit, img)
#     shutil.copyfile(origin, destination)
    
# # test knife regression images
# #10% of data
# imgs = knife_images[5620:6322]
# for img in imgs:
#     origin = os.path.join(data_knife_dir, img)
#     destination = os.path.join(test_profit, img)
#     shutil.copyfile(origin, destination)
    
    
# # validation knife regression images
# #10% of data
# imgs = knife_images[6322:]
# for img in imgs:
#     origin = os.path.join(data_knife_dir, img)
#     destination = os.path.join(val, img)
#     shutil.copyfile(origin, destination)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Conv2D, Dense, Flatten, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization

img_array = cv2.imread('knife_images/918.jpg')  # convert to array

img_rgb = cv2.resize(img_array,(256,256),3)
plt.imshow(img_rgb)  # graph it
plt.show();


In [ ]:
def image_checker(index,):
    img_array = cv2.imread('knife_images/'+str(index)+'.jpg')  
    img_rgb = cv2.resize(img_array,(256,256),3)
    plt.imshow(img_rgb)  # graph it
    plt.show();

In [ ]:
top_benchmade_index[:50]

In [ ]:
image_checker(6158)

In [ ]:
image_checker(2286)

In [ ]:
image_checker(1879)

In [ ]:
image_checker(4326)

In [ ]:
image_checker(6094)

In [ ]:
#final processing steps for images

image_list = []
for x in range(len(df_CNN_regression)):
    
    img_array = cv2.imread('knife_images/'+str(x)+'.jpg')  # convert to array
    img_rgb = cv2.resize(img_array,(256,256),3)  # resize
    img_rgb = np.array(img_rgb).astype(np.float64)/255.0  # scaling
    image_list.append(img_rgb)
   
    # img_rgb = np.expand_dims(img_rgb, axis=0)  # expand dimension



In [ ]:
df_CNN_regression['mean_profit']= (df_CNN_regression['profit']/df_CNN_regression['profit'].mean())

In [ ]:
df_CNN_regression['mean_profit'].describe()

In [ ]:
X = np.array(image_list)

In [ ]:
y=  df_CNN_regression['mean_profit']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, test_size=0.4, random_state=32)# Create the Test and Final Training Datasets

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
print("Xtrain:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

In [ ]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print("Xtrain:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)
print("X_val:", X_test.shape)
print("y_val:", y_test.shape)


In [ ]:
#small batch

# model = models.Sequential()

# model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
#                         input_shape=(256 ,256, 3)))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Flatten())


# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1, activation='linear'))

# model.compile(loss='mean_squared_error',
#               optimizer='Adam',
#                metrics=['mse'])

# history = model.fit(X_train,
#                     y_train,
#                     epochs=30,
#                     batch_size=32,
#                     validation_data=(X_val, y_val))



In [ ]:
results_test = model.evaluate(X_test, y_test)

#model.summary()


In [ ]:
df_scrub['profit'].mean()

In [ ]:
2.2164 * 41.374303202846974

In [ ]:
df_scrub.head()

In [ ]:
model.summary()

In [ ]:
#The model learned patterns wells until epoch 20
#after that the loss spikes signifcantly before dropping again
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot
plt.title('model loss')
plt.ylabel('loss( mean square error)')
plt.xlabel('epoch')
plt.legend(['train_mse', 'val_mse'], loc='upper right')
plt.show();

In [ ]:
model.save('my_model_batch32.h5')

In [ ]:
#a train set of 60% and a val and test size of 20% each 

In [ ]:
#this model showed a lot of indication that it was overfit
#need to retry how I split the data 
#Instead of manul indexing, will use 
# from sklearn model_selection train_test_split



# X_train = X[:4918]
# y_train = y[:4918]

# X_train = X[4918:5971]
# y_train = y[4918:5971]

# X_test = X[5971:]
# y_test = y[5971:]


# display(len(X_val)/len(X))
# display(len(X_train)/len(X))
# len(X_test)/len(X)



# model = models.Sequential()

# model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
#                         input_shape=(224 ,224,  3)))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Flatten())

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.1))

# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))

# model.add(Dense(1, activation='linear'))

# model.compile(loss='mean_squared_error',
#               optimizer='Adam',
#                metrics=['mse'])
# history = model.fit(X_train,
#                     y_train,
#                     epochs=32,
#                     batch_size=300,
#                     validation_data=(X_val, y_val))




# results_train = model.evaluate(X_test, y_test)

#model.summary()


# model.save('my_model_batch500.h5')

In [ ]:
results_train = model.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
# model.save('my_model_batch500.h5')

In [ ]:
history.history.keys()

In [ ]:
#The model is showing a lot of signs of overfitting 
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot
plt.title('model loss')
plt.ylabel('loss( mean square error)')
plt.xlabel('epoch')
plt.legend(['train_mse', 'val_mse'], loc='upper right')
plt.show();

In [ ]:
X_train.shape

In [ ]:
# results_train = model.evaluate(X_test, y_test)

#model.summary()


# model.save('my_model_batch500.h5')

In [ ]:
import time

from ebaysdk.finding import Connection
import requests

from ebaysdk.shopping import Connection as Shopping

import pandas as pd 
import  json
import numpy as np
import re
import preprocess_ddey117 as pp
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image


import seaborn as sns 

df = pd.read_csv('data/tera_df_prepared.csv', sep=',', error_bad_lines=False, index_col=False, dtype='unicode')

df.info()

api = Shopping(config_file='ebay.yaml', debug=True, siteid="EBAY-US")
request = {
           'itemID': list(itemIds),
           'IncludeSelector': 'ItemSpecifics'
          }
response = api.execute('GetMultipleItems', request)

response_dict = response.dict()

results_list_of_dicts = response_dict['Item']



response_dict = response.dict()

    #index dict to appropriate index
# results_list_of_dicts = response_dict['searchResult']['item']

results_list_of_dicts = response_dict['Item']

#create function for organizing API call
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['ItemID', 'Title', 'GalleryURL', 
                'ViewItemURLForNaturalSearch', 'ItemSpecifics', 
                'Country', 'Location', 'ConvertedCurrentPrice',
                'PictureURL']
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
            results.append(prepared_data)
    
       
#         Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
    return results

#create function for organizing API call
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['ItemID', 'Title', 'GalleryURL', 
                'ViewItemURLForNaturalSearch', 'ItemSpecifics', 

                'PictureURL']
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
            results.append(prepared_data)
    
       
#         Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
    return results

def process_list(my_list):
 
    api = Shopping(config_file='ebay.yaml', debug=True, siteid="EBAY-US")
    request = {
               'itemID': my_list,
               'IncludeSelector': 'ItemSpecifics'
              }
    response = api.execute('GetMultipleItems', request)

    

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['Item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)
    
    return full_dataset

full_dataset = []
for i in range(0, len(itemIds), 20):
    process_list(itemIds[i:i+20])

itemIds = list(df.itemId)

full_dataset = []
for i in range(0, len(itemIds), 20):
    process_list(itemIds[i:i+20])

len(full_dataset)

full_dataset[:10]







df2 = pd.DataFrame(full_dataset)

df3 = df2.drop_duplicates(subset='ItemID').copy()

df.info()

df3.ItemSpecifics.sample(5).apply(print)

import re

pattern = re.compile("Opening Mechanism\s*\S+\S+\s*\S+\S+\s\S(\w+)")

def extract_opening_mech(line):
    pattern = re.compile("Opening Mechanism\s*\S+\S+\s*\S+\S+\s\S(\w+)")

    if re.findall(pattern,str(line)):

        match = re.findall(pattern,str(line))[0]

    else:

        match = 'NA'
        
    return match

df3['opening_mechanism'] = df3.ItemSpecifics.apply(extract_opening_mech)

df3

pattern = re.compile("Blade Material\s*\S+\S+\s*\S+\S+\s\S(\w+)")

def extract_blade_type(line):
    pattern = re.compile("Blade Material\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):

        match = re.findall(pattern,str(line))[0]

    else:

        match = 'NA'
        
    return match

df3['blade_type'] = df3.ItemSpecifics.apply(extract_blade_type)

df3.ItemSpecifics.sample(10).apply(print)

def extract_color(line):
    pattern = re.compile("Color\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):

        match = re.findall(pattern,str(line))[0]

    else:

        match = 'NA'
        
    return match

df3['color'] = df3.ItemSpecifics.apply(extract_color)

def extract_blade_type(line):
    pattern = re.compile("Blade Type\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):

        match = re.findall(pattern,str(line))[0]

    else:

        match = 'NA'
        
    return match

df3['blade_type'] = df3.ItemSpecifics.apply(extract_blade_type)

df3['blade_type'].value_counts()[:20]

def extract_manufacture_region(line):
    pattern = re.compile("Country/Region of Manufacture\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):
        match = re.findall(pattern,str(line))[0]
    else:
        match = 'NA'
    return match
        


df3['region_of_Manufacture'] = df3.ItemSpecifics.apply(extract_manufacture_region)

df3['region_of_Manufacture'].value_counts()

def extract_handle_material(line):
    pattern = re.compile("Handle Material\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):
        match = re.findall(pattern,str(line))[0]
    else:
        match = 'NA'
    return match
        


df3['handle_material'] = df3.ItemSpecifics.apply(extract_handle_material)

df3['handle_material'].value_counts()[:50]

def extract_lock_type(line):
    pattern = re.compile("Lock Type\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):
        match = re.findall(pattern,str(line))[0]
    else:
        match = 'NA'
    return match

df3['lock_type'] = df3.ItemSpecifics.apply(extract_lock_type)

df3['lock_type'].value_counts()

def extract_blade_edge(line):
    pattern = re.compile("Blade Edge\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):
        match = re.findall(pattern,str(line))[0]
    else:
        match = 'NA'
    return match
        
df3['blade_edge'] = df3.ItemSpecifics.apply(extract_blade_edge)

df3['blade_edge'].value_counts()

def extract_dexterity(line):
    pattern = re.compile("Dexterity\s*\S+\S+\s*\S+\S+\s\S(\w+)")
    if re.findall(pattern,str(line)):
        match = re.findall(pattern,str(line))[0]
    else:
        match = 'NA'
    return match
        
df3['dexterity'] = df3.ItemSpecifics.apply(extract_dexterity)

df3['dexterity'].value_counts()

df3.to_csv('data/item_specifics_df.csv')







# api = Shopping(config_file='ebay.yaml', debug=True, siteid="EBAY-US")
# request = {
#            'itemID': list(itemIds),
#            'IncludeSelector': 'ItemSpecifics'
#           }
# response = api.execute('GetMultipleItems', request)
# print(response.dict())

ls

from os import listdir
from PIL import Image

for imageFolder in listdir('./nn_images2'):
    try:
        img = Image.open('./nn_images2/'+imageFolder)
        img.verify()     # to veify if its an img
        img.close()     #to close img and free memory space
    except (IOError, SyntaxError) as e:
        print('Bad file:', imageFolder)

listdir('/nnimages2')

root='C:/Users/12108/Documents/GitHub/Neural_Network_Predicting_Reseller_Success_Ebay/nn_images2/'

import pandas as pd
import matplotlib.pyplot  as plt
from PIL import Image
from pathlib import Path
import imagesize
import numpy as np

# Get the Image Resolutions
imgs = [img.name for img in Path(root).iterdir() if img.suffix == ".jpg"]
img_meta = {}
for f in imgs: img_meta[str(f)] = imagesize.get(root+f)

# Convert it to Dataframe and compute aspect ratio
img_meta_df = pd.DataFrame.from_dict([img_meta]).T.reset_index().set_axis(['FileName', 'Size'], axis='columns', inplace=False)
img_meta_df[["Width", "Height"]] = pd.DataFrame(img_meta_df["Size"].tolist(), index=img_meta_df.index)
img_meta_df["Aspect Ratio"] = round(img_meta_df["Width"] / img_meta_df["Height"], 2)

print(f'Total Nr of Images in the dataset: {len(img_meta_df)}')
img_meta_df.head()

# Visualize Image Resolutions

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
points = ax.scatter(img_meta_df.Width, img_meta_df.Height, color='blue', alpha=0.5, picker=True)
ax.set_title("Image Resolution")
ax.set_xlabel("Width", size=14)
ax.set_ylabel("Height", size=14)

# Visualize Image Resolutions

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111)
points = ax.scatter(img_meta_df.Width, img_meta_df.Height, color='blue', alpha=0.5, s=img_meta_df["Aspect Ratio"]*100, picker=True)
ax.set_title("Image Resolution")
ax.set_xlabel("Width", size=14)
ax.set_ylabel("Height", size=14)





#create function for organizing API call
def prepare_data(data_list):
    """
    This function takes in a list of dictionaries and prepares it
    for analysis
    """
    
    # Make a new list to hold results
    results = []
    
    for business_data in data_list:
    
        # Make a new dictionary to hold prepared data for this business
        prepared_data = {}
        
        # Extract name, review_count, rating, and price key-value pairs
        # from business_data and add to prepared_data
        # If a key is not present in business_data, add it to prepared_data
        # with an associated value of None
        
        keys = ['itemId', 'title', 'galleryURL', 
        'viewItemURL', 'autoPay', 'postalCode', 
        'sellingStatus', 'shippingInfo', 'listingInfo',
        'returnsAccepted', 'condition', 'topRatedListing',
        'galleryPlusPictureURL', 'subtitle', 'discountPriceInfo',
        'secondaryCategory']
        
        for key in keys:
            prepared_data[key] = business_data.get(key, None)
            results.append(prepared_data)
    
       
        # Add to list if all values are present
#         if all(prepared_data.values()):
#             results.append(prepared_data)
    
    return results

# def prepare_df(df):
#     price_list = []
#     for row in full_dataset:
#         listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['value'])
#         price_list.append(listed_price)

#     df['price_in_US'] = price_list
#     #pull shipping cost from json dict with regex 
#     df['shipping_cost'] = df['shippingInfo'].apply(lambda x: re.findall("(\d+\S+\d)", json.dumps(x)))
#     df['shipping_cost'] = df['shipping_cost'].apply(lambda x: ''.join(x))
#     df.drop(df[df['shipping_cost'] == ''].index, inplace=True)
#     df['shipping_cost'] = df['shipping_cost'].apply(lambda x: np.float(x))

#     #create new feature 'converted price'
#     df['converted_price'] = df['shipping_cost'] + df['price_in_US']
#     df.drop_duplicates(subset=['itemId'],  keep='first', inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     display(df.head())
#     display(df.info())
#     return df

def prepare_df(df):
    price_list = []
    ship_price_list = []
    condition_list = []
    for row in full_dataset:
        listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['value'])
        price_list.append(listed_price)

        try:
            listed_ship_price = np.float(row['shippingInfo']['shippingServiceCost']['value'])
            ship_price_list.append(listed_ship_price)
        except: 
            listed_ship_price = 0
            ship_price_list.append(listed_ship_price)

        try:
            condition = None
            condition = np.float(row['condition']['conditionId'])
            condition_list.append(condition)
        except: 
            conditon = 0
            condition_list.append(condition)

    df['shipping_cost'] = ship_price_list
    df['price_in_US'] = price_list
    df['condition'] = condition_list
    
    #create new feature 'converted price'
    df['converted_price'] = df['shipping_cost'] + df['price_in_US']
    df.drop_duplicates(subset=['itemId'],  keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    display(df.head())
    display(df.info())
    return df

overhead_cost = 3
def prepare_brands(df, bucket_dict_position):

    df.title = df.title.apply(str.lower)

    #remove special characters
    df.title.apply(pp.remove_special_chars)

    df[str(list(bucket_dict.keys())[bucket_dict_position])] = float(list(bucket_dict.values())[bucket_dict_position])
    
    df['profit'] = (df['converted_price'] - df[list(bucket_dict.keys())[bucket_dict_position]] - overhead_cost)
    df['ROI'] = (df['profit']/(df[list(bucket_dict.keys())[bucket_dict_position]] + overhead_cost))*100.0
    
    return df

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from base64 import b64encode




#this function was used at first to check accuracy of calls using ebay API
#the less complex "prepare_brands" function seems sufficient enough
#it is redudant to use regex with working ebay API call

def modified_prepare_brands(df, bucket_dict_position):

    df.title = df.title.apply(str.lower)

    #remove special characters
    df.title.apply(pp.remove_special_chars)

    pattern = re.compile(list(bucket_dict.keys())[bucket_dict_position])
    df[list(bucket_dict.keys())[bucket_dict_position]] = df_bench.title.apply(lambda x: re.sub(pattern, x, str(list(bucket_dict.values())[bucket_dict_position])))

    df[list(bucket_dict.keys())[bucket_dict_position]] = df[list(bucket_dict.keys())[bucket_dict_position]].apply(lambda x: np.float(x))
    
    df['profit'] = (df['converted_price'] - df[list(bucket_dict.keys())[bucket_dict_position]] - overhead_cost)
    df['ROI'] = (df['profit']/(df[list(bucket_dict.keys())[bucket_dict_position]] + overhead_cost))*100.0

    return df

request = {
           'categoryId': 48818,
            'itemFilter': [
                            {'name': 'Condition', 'value': 'Used'},
                            {'name': 'ListingType', 'value': 'FixedPrice'}
                          ],
            'aspectFilter': [
                              {'aspectName': 'Brand', 'aspectValueName': 'Benchmade'},
                              {'aspectName': 'Brand', 'aspectValueName': 'Buck'},
                              {'aspectName': 'Brand', 'aspectValueName': 'Case'},
                              {'aspectName': 'Brand', 'aspectValueName': 'Kershaw'},
                              {'aspectName': 'Brand', 'aspectValueName': 'Victorinox'},
                              {'aspectName': 'Brand', 'aspectValueName': 'CRKT'},
                              {'aspectName': 'Brand', 'aspectValueName': 'Leatherman'},
                              {'aspectName': 'Brand', 'aspectValueName': 'Spyderco'},
                             ],
            'paginationInput': {
                                'entriesPerPage': 100,
                                'pageNumber': 1

                                }}

# Create an empty list for the full prepared dataset
full_dataset = []
#debugger error at page 23


for page in range(1, 22):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'categoryId': 48818,
                'itemFilter': [
#                                 {'name': 'Condition', 'value': 'Used'},
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [
                                  {'aspectName': 'Brand', 'aspectValueName': 'Benchmade'}],

                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_bench = pd.DataFrame(full_dataset)

bucket_dict = {'benchmade': 45.0,
               'buck': 20.0,
               'case': 20.0,
               'crkt': 15.0,
               'kershaw': 15.0,
               'leatherman': 30.0, 
               'spyderco': 30.0,
               'victorinox': 20.0
              }

df_bench = prepare_df(df_bench)

df_bench = prepare_brands(df_bench, 0)

df_bench['benchmade'].value_counts()

df_bench.info()

df_bench[df_bench['galleryPlusPictureURL'].notna()]['galleryPlusPictureURL'].sample(30).apply(print)

df_bench['profit'].describe()

df_bench['ROI'].describe()

df_bench.to_csv('data/df_bench.csv', index=False)

### Domain Understading: Cost Breakdown
- padded envelopes: \$0.50 per knife
- flatrate shipping: \$4.45 per knife
- brand knife at surplus store: 15, 20, 30, or 45 dollars per knife
- overhead expenses (gas, cleaning suplies, sharpening supplies, etc): $7

# Create an empty list for the full prepared dataset
full_dataset = []
#debugger error at page 42


for page in range(1, 41):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [
                                 {'aspectName': 'Brand', 'aspectValueName': 'Buck'}],

                    
                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_buck = pd.DataFrame(full_dataset)

df_buck = prepare_df(df_buck)

df_buck = prepare_brands(df_buck, 1)

df_buck.info()

df_buck['buck'].value_counts()

df_buck['profit'].describe()

df_buck['ROI'].describe()

df_buck.to_csv('data/df_buck.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []



for page in range(1, 100):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [
#                               
                                   {'aspectName': 'Brand', 'aspectValueName': 'Case XX'},
                                   {'aspectName': 'Brand', 'aspectValueName': 'Case'}
                                 ],

                    
                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_case = pd.DataFrame(full_dataset)

df_case = prepare_df(df_case)

df_case = prepare_brands(df_case, 2)

df_case.info()

df_case.to_csv('data/df_case.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []
#debug error page 18


for page in range(1, 17):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [

                                    {'aspectName': 'Brand', 'aspectValueName': 'CRKT'}],
                 
                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_crkt = pd.DataFrame(full_dataset)

df_crkt = prepare_df(df_crkt)

bucket_dict 

df_crkt = prepare_brands(df_crkt, 3)

display(df_crkt.head())
display(df_crkt.info())

df_crkt.to_csv('data/df_crkt.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []
#debugger threw error at page 70


for page in range(1, 69):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [

                                  {'aspectName': 'Brand', 'aspectValueName': 'Kershaw'}
                                ],

                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_kershaw = pd.DataFrame(full_dataset)

bucket_dict

df_kershaw = prepare_df(df_kershaw)

df_kershaw = prepare_brands(df_kershaw, 4)

df_kershaw.info()

df_kershaw.to_csv('data/df_kershaw.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []
#debug error page 18


for page in range(1, 17):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [

                                   {'aspectName': 'Brand', 'aspectValueName': 'Leatherman'}],
                    
                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_leatherman = pd.DataFrame(full_dataset)

bucket_dict

df_leatherman = prepare_df(df_leatherman)

df_leatherman = prepare_brands(df_leatherman, 5)

df_leatherman.info()

df_leatherman.to_csv('data/df_leatherman.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []
#debug error page 72

for page in range(1, 71):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [

                                   {'aspectName': 'Brand', 'aspectValueName': 'Spyderco'}
                              ],
                    
                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))

df_spyderco = pd.DataFrame(full_dataset)

df_spyderco = prepare_df(df_spyderco)

bucket_dict 

df_spyderco = prepare_brands(df_spyderco, 6)

df_spyderco.info()

df_spyderco.to_csv('data/df_spyderco.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []


for page in range(1, 21):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [

                                   {'aspectName': 'Brand', 'aspectValueName': 'SOG'}],

                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_sog = pd.DataFrame(full_dataset)

df_sog = prepare_df(df_sog)

bucket_dict['sog'] = 15.0

df_sog = prepare_brands(df_sog, 8)

df_sog.info()

df_sog.to_csv('data/df_sog.csv', index=False)

# Create an empty list for the full prepared dataset
full_dataset = []


for page in range(1, 100):
    # Add or update the "offset" key-value pair in url_params
    request['paginationInput']['pageNumber'] = page
    
    # Make the query and get the response

    api = Connection(config_file='ebay.yaml', debug=True, siteid="EBAY-US")

    request = {
                'keywords': 'knife',
                'itemFilter': [
                                {'name': 'ListingType', 'value': 'FixedPrice'}
                              ],
                'aspectFilter': [

                                   {'aspectName': 'Brand', 'aspectValueName': 'Victorinox'}],

                    
                'paginationInput': {
                                    'entriesPerPage': 100,
                                    'pageNumber': page
                    
                                    },
               
                }

    response = api.execute('findItemsAdvanced', request)

    #save the response as a json dict
    response_dict = response.dict()



    #index dict to appropriate index
    results_list_of_dicts = response_dict['searchResult']['item']

    # Call the prepare_data function to get a list of processed data
    prepared_knives = prepare_data(results_list_of_dicts)

    # Extend full_dataset with this list (don't append, or you'll get
    # a list of lists instead of a flat list)
    full_dataset.extend(prepared_knives)

# Check the length of the full dataset. It will be up to `total`,
# potentially less if there were missing values
display(len(full_dataset))


df_victorinox = pd.DataFrame(full_dataset)

df_victorinox = prepare_df(df_victorinox)

bucket_dict

df_victorinox = prepare_brands(df_victorinox, 7)

df_victorinox.info()

df_victorinox.to_csv('data/df_victorinox.csv', index=False)

# df_bench = pd.read_csv("data/df_bench.csv")
# df_buck = pd.read_csv("data/df_buck.csv")
# df_case = pd.read_csv("data/df_case.csv")
# df_crkt = pd.read_csv("data/df_crkt.csv")
# df_kershaw = pd.read_csv("data/df_kershaw.csv")
# df_leatherman = pd.read_csv("data/df_leatherman.csv")
# df_spyderco = pd.read_csv("data/df_spyderco.csv")
# df_victorinox = pd.read_csv("data/df_victorinox.csv")

df = pd.concat([df_bench,df_buck,df_case,df_crkt,df_kershaw,df_leatherman,df_spyderco,df_sog,df_victorinox])

df.info()

df.isna().sum()

df.to_csv("data/full_dataset2.csv", index=False)

After succesfully going through 10,000 items on ebay's website and extracting everything possible, there is still a little bit more extracting to do from the json dictionary before saving the dataframe again. We need to get the price of the knives out of the nested dictionary in the dataframe as well as the shipping cost. After that, I would like to create a new feature called
**"converted price," which is simply the price of the knife listed on the ebay's website plus shipping.**

```
#Create row for converted Price of Knives in US dollars
price_list = []
for row in full_dataset:
    listed_price = np.float(row['sellingStatus']['convertedCurrentPrice']['value'])
    price_list.append(listed_price)
    
df['price_in_US'] = price_list
```

```
#atttempt to pull shipping cost from json dict
shipping_cost_list = []
for row in full_dataset:
    shipping_cost = np.float(row['shippingInfo']['shippingServiceCost']['value'])
    shipping_cost_list.append(shipping_cost)
    
df['shipping_price'] = shipping_cost_list
```

```
#pull shipping cost from json dict with regex 
df['shipping_cost'] = df['shippingInfo'].apply(lambda x: re.findall("(\d+\S+\d)", json.dumps(x)))
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: ''.join(x))
df.drop(df[df['shipping_cost'] == ''].index, inplace=True)
df['shipping_cost'] = df['shipping_cost'].apply(lambda x: np.float(x))

#create new feature 'converted price'
df['converted_price'] = df['shipping_cost'] + df['price_in_US']
df = df.drop_duplicates(subset=['title', 'galleryURL'], keep='first')
display(df.head())
display(df.info())

df.to_csv('data/full_dataset.csv', index=False)
```

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***


# here you run your code to clean the data

```
import code.data_cleaning as dc

full_dataset = dc.full_clean()
```

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***
# here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***


## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***



# data_knife_dir = 'knife_images'
# data_profit_dir = 'data/profit'
# new_dir = 'split'

# os.mkdir(new_dir)

# train_folder = os.path.join(new_dir, 'train')
# train_profit = os.path.join(train_folder, 'profit')
# os.mkdir(train_folder)
# os.mkdir(train_profit)

# test_folder = os.path.join(new_dir, 'test')
# test_profit = os.path.join(test_folder, 'profit')
# os.mkdir(test_folder)
# os.mkdir(test_profit)


# val_folder = os.path.join(new_dir, 'validation')
# val_profit = os.path.join(val_folder, 'profit')
# os.mkdir(val_folder)
# os.mkdir(val_profit)

# val_profit

# # train knife regression images
# #80% of data
# imgs = knife_images[:5620]
# for img in imgs:
#     origin = os.path.join(data_knife_dir, img)
#     destination = os.path.join(train_profit, img)
#     shutil.copyfile(origin, destination)
    
# # test knife regression images
# #10% of data
# imgs = knife_images[5620:6322]
# for img in imgs:
#     origin = os.path.join(data_knife_dir, img)
#     destination = os.path.join(test_profit, img)
#     shutil.copyfile(origin, destination)
    
    
# # validation knife regression images
# #10% of data
# imgs = knife_images[6322:]
# for img in imgs:
#     origin = os.path.join(data_knife_dir, img)
#     destination = os.path.join(val, img)
#     shutil.copyfile(origin, destination)

import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Conv2D, Dense, Flatten, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization

img_array = cv2.imread('knife_images/918.jpg')  # convert to array

img_rgb = cv2.resize(img_array,(256,256),3)
plt.imshow(img_rgb)  # graph it
plt.show();


def image_checker(index,):
    img_array = cv2.imread('knife_images/'+str(index)+'.jpg')  
    img_rgb = cv2.resize(img_array,(256,256),3)
    plt.imshow(img_rgb)  # graph it
    plt.show();

top_benchmade_index[:50]

image_checker(6158)

image_checker(2286)

image_checker(1879)

image_checker(4326)

image_checker(6094)

#final processing steps for images

image_list = []
for x in range(len(df_CNN_regression)):
    
    img_array = cv2.imread('knife_images/'+str(x)+'.jpg')  # convert to array
    img_rgb = cv2.resize(img_array,(256,256),3)  # resize
    img_rgb = np.array(img_rgb).astype(np.float64)/255.0  # scaling
    image_list.append(img_rgb)
   
    # img_rgb = np.expand_dims(img_rgb, axis=0)  # expand dimension



df_CNN_regression['mean_profit']= (df_CNN_regression['profit']/df_CNN_regression['profit'].mean())

df_CNN_regression['mean_profit'].describe()

X = np.array(image_list)

y=  df_CNN_regression['mean_profit']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, test_size=0.4, random_state=32)# Create the Test and Final Training Datasets

X.shape

y.shape

print("Xtrain:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

print("Xtrain:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)
print("X_val:", X_test.shape)
print("y_val:", y_test.shape)




#small batch

# model = models.Sequential()

# model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
#                         input_shape=(256 ,256, 3)))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Flatten())


# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))
# model.add(Dense(1, activation='linear'))

# model.compile(loss='mean_squared_error',
#               optimizer='Adam',
#                metrics=['mse'])

# history = model.fit(X_train,
#                     y_train,
#                     epochs=30,
#                     batch_size=32,
#                     validation_data=(X_val, y_val))



results_test = model.evaluate(X_test, y_test)

#model.summary()


df_scrub['profit'].mean()

2.2164 * 41.374303202846974

df_scrub.head()



model.summary()

#The model learned patterns wells until epoch 20
#after that the loss spikes signifcantly before dropping again
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot
plt.title('model loss')
plt.ylabel('loss( mean square error)')
plt.xlabel('epoch')
plt.legend(['train_mse', 'val_mse'], loc='upper right')
plt.show();

model.save('my_model_batch32.h5')





#a train set of 60% and a val and test size of 20% each 

#this model showed a lot of indication that it was overfit
#need to retry how I split the data 
#Instead of manul indexing, will use 
# from sklearn model_selection train_test_split



# X_train = X[:4918]
# y_train = y[:4918]

# X_train = X[4918:5971]
# y_train = y[4918:5971]

# X_test = X[5971:]
# y_test = y[5971:]


# display(len(X_val)/len(X))
# display(len(X_train)/len(X))
# len(X_test)/len(X)



# model = models.Sequential()

# model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu',
#                         input_shape=(224 ,224,  3)))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

# model.add(layers.Flatten())

# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.1))

# model.add(Dense(256, activation='relu'))
# model.add(Dense(128, activation='relu'))

# model.add(Dense(1, activation='linear'))

# model.compile(loss='mean_squared_error',
#               optimizer='Adam',
#                metrics=['mse'])
# history = model.fit(X_train,
#                     y_train,
#                     epochs=32,
#                     batch_size=300,
#                     validation_data=(X_val, y_val))




# results_train = model.evaluate(X_test, y_test)

#model.summary()


# model.save('my_model_batch500.h5')

results_train = model.evaluate(X_test, y_test)

model.summary()

# model.save('my_model_batch500.h5')

history.history.keys()

#The model is showing a lot of signs of overfitting 
fig = plt.figure(figsize=(12,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot
plt.title('model loss')
plt.ylabel('loss( mean square error)')
plt.xlabel('epoch')
plt.legend(['train_mse', 'val_mse'], loc='upper right')
plt.show();

X_train.shape

# results_train = model.evaluate(X_test, y_test)

#model.summary()


# model.save('my_model_batch500.h5')

